In [267]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 9, in <module>
  File "fmqttqueue.py", line 1, in <module>
  File "mqtt_as.py", line 28, in <module>
KeyboardInterrupt: 
MicroPython v1.10-227-g696549d2e on 2019-03-17; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [268]:
%sendtofile --source minimalmdns.py

Sent 65 lines (2256 bytes) to minimalmdns.py.


In [269]:
import network, time
si = network.WLAN(network.STA_IF) # create access-point interface
si.active(True)         # activate the interface
si.connect("DoESLiverpool", "decafbad00")
while not si.isconnected():
    time.sleep(0.1)


.

In [270]:
print(si.isconnected())

True


In [272]:
from minimalmdns import mdnshostnametoipnumber
hostname = "mqtt.local"
hostipnumber = mdnshostnametoipnumber(si, hostname)
print(hostname, hostipnumber)


mqtt.local 10.0.30.194


In [168]:
import socket
print(socket.getaddrinfo("freesteel.co.uk", 8080))


[(2, 1, 0, 'freesteel.co.uk', ('93.93.131.30', 8080))]


In [169]:
print(socket.getaddrinfo("mqtt.local", 8080))


[]


In [253]:
import socket, select, time, ustruct

def createmdnsrequestpacket(name):
    q = bytearray(b'\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00')
    for x in name.encode().split(b"."):
        q.append(len(x))
        q.extend(x)
    q.extend(b"\x00\x00\x01\x00\x01")
    return q

def extractpackedname(buf, o):
    names = [ ]
    while buf[o] != 0:
        if buf[o] & 0xc0:
            o = ustruct.unpack_from("!H", buf, o)[0] & 0x3FFF
        else:
            names.append(bytes(buf[o+1:o+1+buf[o]]))
            o += 1+buf[o]
    return b".".join(names).decode()

def lenpackedname(buf, o):
    i = 0
    while (buf[o+i] != 0) and ((buf[o+i] & 0xc0) != 0xc0):
        i += 1 + buf[o+i]
    return i + (1 if buf[o+i] == 0 else 2)

def decoderesponsepacket(name, buf):
    hostipnumber = ''
    try:
        pkt_id, flags, qst_count, ans_count = ustruct.unpack_from("!HHHH", buf)
        o = 12
        for i in range(qst_count):
            o += lenpackedname(buf, o) + 4
        for i in range(ans_count):
            l = lenpackedname(buf, o)
            if name == extractpackedname(buf, o):
                hostipnumber = ".".join(map(str, buf[o+l+10:o+l+14]))
            o += 10 + l + ustruct.unpack_from("!H", buf, o+l+8)[0]
    except IndexError:
        print("Index error processing packet; probably malformed data")
    return hostipnumber



In [257]:
def decodehostname(name):
    MDNS_ADDR = '224.0.0.251'
    MDNS_PORT = const(5353)

    ipnumber = si.ifconfig()[0]
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    member_info = bytes(tuple(map(int, MDNS_ADDR.split("."))) + tuple(map(int, ipnumber.split("."))))
    sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, member_info)
    
    for i in range(3):
        q = createmdnsrequestpacket(name)
        sock.sendto(q, (MDNS_ADDR, MDNS_PORT))
        rr = select.select([sock], [], [], 1)[0]
        while rr:
            buf, addr = sock.recvfrom(250)
            if addr[0] != ipnumber:
                hostipnumber = decoderesponsepacket(hostname, buf)
                if hostipnumber:
                    return hostipnumber
            rr = select.select([sock], [], [], 0)[0]
    return ''

print(decodehostname(hostname))

10.0.30.194


In [250]:
buf, addr = sock.recvfrom(500)
print(buf, addr)


b'\x00\x01\x84\x00\x00\x01\x00\x01\x00\x00\x00\x00\x04mqtt\x05local\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2' ('10.0.30.194', 5353)


In [254]:
print(decoderesponsepacket(hostname, buf))

10.0.30.194


In [171]:
ipnumber = si.ifconfig()[0]
member_info = bytes((223, 0, 0, 251) + tuple(map(int, ipnumber.split("."))))
print(member_info)
                    

b'\xdf\x00\x00\xfb\n\x00%\xed'


In [247]:

hostname = "mqtt.local"

MDNS_ADDR = '224.0.0.251'
MDNS_PORT = const(5353)

ipnumber = si.ifconfig()[0]
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
member_info = bytes(tuple(map(int, MDNS_ADDR.split("."))) + tuple(map(int, ipnumber.split("."))))
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, member_info)



In [84]:
def check_name(n):
    if isinstance(n, str):
        n = n.split(".")
        if n[-1] == '':
            n = n[:-1]
    n = [i.encode("UTF8") if isinstance(i, str) else i for i in n]
    return n
print(check_name("mqtt.locadd.dl"))

[b'mqtt', b'locadd', b'dl']


In [42]:
# Pack a question into a new array and return it as a memoryview
def pack_question(name, qtype, qclass):
    # Return a pre-packed question as a memoryview
    name = check_name(name)
    name_len = name_packed_len(name)
    buf = bytearray(name_len + 4)
    pack_name(buf, name)
    pack_into("!HH", buf, name_len, qtype, qclass)
    return memoryview(buf)


In [177]:
# scavenged from https://github.com/nickovs/slimDNS
hostname = "mqtt.local"
_MDNS_PORT = const(5353);
MAX_PACKET_SIZE = const(1024)
_CLASS_MASK   = const(0x7FFF)
local_addr = ipnumber

from ustruct import pack_into, unpack_from
from select import select
import time

def pack_name(buf, name):
    # We don't support writing with name compression, BIWIOMS
    o = 0
    for part in name:
        pl = len(part)
        buf[o] = pl
        buf[o+1:o+pl+1] = part
        o += pl+1
    buf[o] = 0
    
# Find the memory size needed to pack a name without compression
def name_packed_len(name):
    return sum(len(i)+1 for i in name) + 1
    
# Pack a question into a new array and return it as a memoryview
def pack_question(name, qtype, qclass):
    # Return a pre-packed question as a memoryview
    name = check_name(name)
    name_len = name_packed_len(name)
    buf = bytearray(name_len + 4)
    pack_name(buf, name)
    pack_into("!HH", buf, name_len, qtype, qclass)
    return memoryview(buf)

# Move the offset past the name to which it currently points
def skip_name_at(buf, o):
    while True:
        l = buf[o]
        if l == 0:
            o += 1
            break
        elif (l & 0xc0) == 0xc0:
            o += 2
            break
        else:
            o += l+1
    return o


# Ensure that a name is in the form of a list of encoded blocks of
# bytes, typically starting as a qualified domain name
def check_name(n):
    if isinstance(n, str):
        n = n.split(".")
        if n[-1] == '':
            n = n[:-1]
    n = [i.encode("UTF8") if isinstance(i, str) else i for i in n]
    return n

_TYPE_A    = const(1)
_CLASS_IN     = const(1)


import time

answers = []

# Advance the offset past the question to which it points
def skip_question(buf, o):
    o = skip_name_at(buf, o)
    return o + 4

# Advance the offset past the answer to which it points
def skip_answer(buf, o):
    o = skip_name_at(buf, o)
    (rdlen,) = unpack_from("!H", buf, o+8)
    return o + 10 + rdlen

# Test if two possibly compressed names are equal
def compare_packed_names(buf, o, packed_name, po=0):
    while packed_name[po] != 0:
        print("pp1", o, po)
        while buf[o] & 0xc0:
            (o,) = unpack_from("!H", buf, o)
            o &= 0x3fff
        while packed_name[po] & 0xc0:
            (po,) = unpack_from("!H", packed_name, po)
            po &= 0x3fff
        print("pp2", o, po)
        l1 = buf[o] +1
        l2 = packed_name[po] +1
        if l1 != l2 or buf[o:o+l1] != packed_name[po:po+l2]:
            return False
        o += l1
        po += l2
    return buf[o] == 0


def compare_q_and_a(q_buf, q_offset, a_buf, a_offset=0):
    print("compare_q_and_a", bytes(q_buf), q_offset, bytes(a_buf), a_offset)
    if not compare_packed_names(q_buf, q_offset, a_buf, a_offset):
        return False
    print("compare_q_and_a", bytes(q_buf[q_offset:]), q_offset, bytes(a_buf[a_offset:]), a_offset)
    (q_type, q_class) = unpack_from("!HH", q_buf, skip_name_at(q_buf, q_offset))
    (r_type, r_class) = unpack_from("!HH", a_buf, skip_name_at(a_buf, a_offset))
    if not (q_type == r_type or q_type == _TYPE_ANY):
        return False
    q_class &= _CLASS_MASK
    r_class &= _CLASS_MASK
    return (q_class == r_class or q_class == _TYPE_ANY)


_reply_buffer = None
def process_packet(buf, addr):
    global _reply_buffer
    # Process a single multicast DNS packet
    (pkt_id, flags, qst_count, ans_count, _, _) = unpack_from("!HHHHHH", buf, 0)
    o = 12
    matches = []
    reply_len = 12
    for i in range(qst_count):
        print("qcount", o)
        o = skip_question(buf, o)
        print("qcountA", o)

    # In theory we could do known answer suppression here
    # We don't, BIWIOMS
    if _pending_question:
        print("ans_count", ans_count, o)
        for i in range(ans_count):
            if compare_q_and_a(_pending_question, 0, buf, o):
                print("got answer", bytes(buf[o:skip_answer(buf,o)]))
                if answers.append(buf[o:skip_answer(buf,o)]):
                    answered = True
            o = skip_answer(buf,o)

    if not matches:
        return

    # We could check for duplicates in the answers (which is
    # possible) but we don't, BIWIOMS

    # Since Micropython sockets don't currently support
    # recvfrom_into() we need to have our own buffer for the
    # reply, even though we are now done with the receiving buffer

    if not _reply_buffer or len(_reply_buffer) < reply_len:
        # print("Making new reply buffer of len {}".format(reply_len))
        _reply_buffer = memoryview(bytearray(reply_len))

    buf = _reply_buffer
    pack_into("!HHHHHH", buf, 0,
              pkt_id, _FLAGS_QR_RESPONSE | _FLAGS_AA,
              0, len(matches), 0, 0)
    o = 12
    for a in matches:
        l = len(a)
        buf[o:o+l] = a
        o += l

    # print("Sending packed reply: {}".format(bytes(buf[:o])))

    # We fake the handling of unicast replies. If the packet came
    # from the mutlicast port we multicast the reply but if it
    # came from any other port we unicast the reply.
    self.sock.sendto(buf[:o], (_MDNS_ADDR, _MDNS_PORT) if addr[0] == _MDNS_PORT else addr)



def process_waiting_packets():
    # Handle all the packets that can be read immediately and
    # return as soon as none are waiting
    while True:
        readers, _, _ = select([sock], [], [], 0)
        if not readers:
            break
        buf, addr = sock.recvfrom(MAX_PACKET_SIZE)
        print("packet received")
        print(buf)
        # print("Received {} bytes from {}".format(len(buf), addr))
        if buf and addr[0] != local_addr:
            try:
                process_packet(memoryview(buf), addr)
            except IndexError:
                print("Index error processing packet; probably malformed data")
            except Exception as e:
                print("Error processing packet: {}".format(e))
                # raise e

_pending_question = None
_answer_callback = None
answered = None




# Look up an IPv4 address for a hostname using mDNS.
q = pack_question(hostname, _TYPE_A, _CLASS_IN)
_pending_question = q
print(bytes(q))

#def _answer_handler(a):
#    addr_offset = skip_name_at(a, 0) + 10
#    answer.append(a[addr_offset:addr_offset+4])
#    return True
#return bytes(answer[0]) if answer else None

fast=False
retry_count=3

# Send our a (packed) question, and send matching replies to
# the answer_callback function.  This will stop after sending
# the given number of retries and waiting for the a timeout on
# each, or sooner if the answer_callback function returns True
p = bytearray(len(q)+12)
pack_into("!HHHHHH", p, 0, 1, 0, 1, 0, 0, 0, 0)
p[12:] = q

try:
    for i in range(retry_count):
        if answered:
            break
        sock.sendto(p, (MDNS_ADDR, MDNS_PORT))
        timeout = time.ticks_ms() + (250 if fast else 1000)
        while not answered:
            sel_time = time.ticks_diff(timeout, time.ticks_ms())
            if sel_time <= 0:
                break
            (rr, _, _) = select([sock], [], [], sel_time/1000.0)
            print(rr)
            if rr:
                process_waiting_packets()
finally:
    _pending_question = None
    _answer_callback = None

for a in answers:
    addr_offset = skip_name_at(a, 0) + 10
    answer = a[addr_offset:addr_offset+4]
    print(list(answer))
    

b'\x04mqtt\x05local\x00\x00\x01\x00\x01'
[<socket>]
packet received
b'\x00\x01\x84\x00\x00\x01\x00\x01\x00\x00\x00\x00\x04mqtt\x05local\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2'
qcount 12
qcountA 28
ans_count 1 28
compare_q_and_a b'\x04mqtt\x05local\x00\x00\x01\x00\x01' 0 b'\x00\x01\x84\x00\x00\x01\x00\x01\x00\x00\x00\x00\x04mqtt\x05local\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2' 28
pp1 0 28
pp2 0 12
pp1 5 17
pp2 5 17
compare_q_and_a b'\x04mqtt\x05local\x00\x00\x01\x00\x01' 0 b'\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2' 28
got answer b'\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2'
[]
[<socket>]
packet received
b'\x00\x01\x84\x00\x00\x01\x00\x01\x00\x00\x00\x00\x04mqtt\x05local\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2'
qcount 12
qcountA 28
ans_count 1 28
compare_q_and_a b'\x04mqtt\x05local\x00\x00\x01\x00\x01' 0 b'\x00\x01\x84\x00\x00\x0

In [64]:
print(bytes(answers[2]))

b'\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2'


In [69]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [174]:
import socket

hostname = "mqtt.local"

MDNS_ADDR = '224.0.0.251'
MDNS_PORT = const(5353)

ipnumber = si.ifconfig()[0]
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
member_info = bytes(tuple(map(int, MDNS_ADDR.split("."))) + tuple(map(int, ipnumber.split("."))))
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, member_info)


In [243]:
hostname = "mqtt.local"

def createmdnsrequestpacket(name):
    q = bytearray(b'\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00')
    for x in name.encode().split(b"."):
        q.append(len(x))
        q.extend(x)
    q.extend(b"\x00\x00\x01\x00\x01")
    return q

def extractpackedname(buf, o):
    names = [ ]
    while buf[o] != 0:
        if buf[o] & 0xc0:
            o = ustruct.unpack_from("!H", buf, o)[0] & 0x3FFF
        else:
            names.append(bytes(buf[o+1:o+1+buf[o]]))
            o += 1+buf[o]
    print(names)
    return b".".join(names).decode()

def lenpackedname(buf, o):
    i = 0
    while (buf[o+i] != 0) and ((buf[o+i] & 0xc0) != 0xc0):
        i += 1 + buf[o+i]
    return i + (1 if buf[o+i] == 0 else 2)

def decoderesponsepacket(hostname, buf):
    hostipnumber = ''
    try:
        pkt_id, flags, qst_count, ans_count = ustruct.unpack_from("!HHHH", buf)
        o = 12
        for i in range(qst_count):
            o += lenpackedname(buf, o) + 4
        for i in range(ans_count):
            l = lenpackedname(buf, o)
            if hostname == extractpackedname(buf, o):
                hostipnumber = ".".join(map(str, buf[o+l+10:o+l+14]))
                print(hostipnumber)
            o += 10 + l + ustruct.unpack_from("!H", buf, o+l+8)[0]
        print(o, len(buf))
    except IndexError:
        print("Index error processing packet; probably malformed data")
    return hostipnumber



In [244]:
import time
import select
q = createmdnsrequestpacket(hostname)
sock.sendto(q, (MDNS_ADDR, MDNS_PORT))
rr = select.select([sock], [], [], 1)[0]
print(rr)

[<socket>]


In [245]:
buf, addr = sock.recvfrom(500)
print(buf, addr)

b'\x00\x01\x84\x00\x00\x01\x00\x01\x00\x00\x00\x00\x04mqtt\x05local\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00\x00\n\x00\x04\n\x00\x1e\xc2' ('10.0.30.194', 5353)


In [142]:
print(ipnumber, addr[0])

10.0.37.237 10.0.30.194


In [139]:

# print("Received {} bytes from {}".format(len(buf), addr))
if buf and addr[0] != local_addr:
    try:
        process_packet(memoryview(buf), addr)
    except IndexError:
        print("Index error processing packet; probably malformed data")
    except Exception as e:
        print("Error processing packet: {}".format(e))
        # raise e


[<socket>]


In [159]:
print(ustruct.unpack_from("!HHHHHH", buf, 0))

(1, 33792, 1, 1, 0, 0)


In [240]:

import ustruct

pkt_id, flags, qst_count, ans_count = ustruct.unpack_from("!HHHH", buf)

hostipnumber = ''
o = 12
for i in range(qst_count):
    o += lenpackedname(buf, o) + 4
for i in range(ans_count):
    l = lenpackedname(buf, o)
    if hostname == extractpackedname(buf, o):
        hostipnumber = ".".join(map(str, buf[o+l+10:o+l+14]))
        print(hostipnumber)
    o += 10 + l + ustruct.unpack_from("!H", buf, o+l+8)[0]
print(o, len(buf))

#except IndexError:
#    print("Index error processing packet; probably malformed data")


[b'mqtt', b'local']
10.0.30.194
44 44
